In [ ]:
!pip install rake-nltk
!python -m spacy download es_core_news_md
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-01-30 12:56:40.409886: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 15.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from textblob import TextBlob
import statistics
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import string
import spacy
from spacy.lang.es.examples import sentences 
from spacy.lang.es.stop_words import STOP_WORDS
from sklearn.decomposition import LatentDirichletAllocation
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from textblob import TextBlob
import re


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
nltk.download('stopwords');
nltk.download('wordnet');
nltk.download('punkt');
nltk.download('omw-1.4');

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
df = pd.read_csv('data_clean_145 - data_clean_145.csv')
df2 = pd.read_csv('data_clean_desde_indice_1078_hasta_el_final_csv.csv')
df2.rename(columns = {'CAMPA��':'CAMPAÑA'}, inplace = True)
df2.rename(columns = {'Valido':'Valida'}, inplace = True)
df2.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1','Unnamed: 0.1.1.1.1', 'Unnamed: 0.1.1.1.1.1', 'level_0', 'index','Unnamed: 0.1.1.1.1.1.1'], inplace=True, axis=1)


In [ ]:
df2.columns

Index(['ID_MOPC', 'IDMOROSO', 'IDCLIENTE', 'OPERADOR', 'CAMPAÑA',
       'IDGRABACION', 'TIPO_DISCADOR', 'FECHA', 'ID', 'IDLLAMADA', 'ESTADO',
       'SUBESTADO', 'F_INICIO', 'ANI_TELEFONO', 'GRABACION_DEP', 'TIPO', 'CRM',
       'BASE', 'LOG', 'DIRECCION', 'IDOPERADOR', 'AGENDAMIENTO_ESTADO', 'anos',
       'mes', 'transcripcion_limpia', 'transcripcion', 'Contestador',
       'Valida'],
      dtype='object')

In [ ]:
df.columns

Index(['ID_MOPC', 'IDMOROSO', 'IDCLIENTE', 'OPERADOR', 'CAMPAÑA',
       'IDGRABACION', 'TIPO_DISCADOR', 'FECHA', 'ID', 'IDLLAMADA', 'ESTADO',
       'SUBESTADO', 'F_INICIO', 'ANI_TELEFONO', 'GRABACION_DEP', 'TIPO', 'CRM',
       'BASE', 'LOG', 'DIRECCION', 'IDOPERADOR', 'AGENDAMIENTO_ESTADO', 'anos',
       'mes', 'transcripcion_limpia', 'Contestador', 'transcripcion',
       'Valida'],
      dtype='object')

In [ ]:
len(df), len(df2) #1870

(2157, 1871)

In [ ]:
for n in range(len(df)):
  for i in range(len(df2)):
    if df.IDGRABACION[n] == df2.IDGRABACION[i]:
      df2.transcripcion[i] = df.transcripcion[n]

<ipython-input-8-ff9db34c7d37>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.transcripcion[i] = df.transcripcion[n]


In [ ]:
df2.transcripcion[134]

"['[00:00.000 --> 00:01.000]  Hola.', '[00:01.000 --> 00:04.000]  Sí, buen día, la servista Yagelín.', '[00:04.000 --> 00:07.000]  No, no se encuentra ella.', '[00:07.000 --> 00:12.000]  Eh, señor, la está llamando a la señora María Rosa de la Agencia M.O.P.C.', '[00:12.000 --> 00:16.000]  por la dedo del Banco Francesco. ¿Usted habló el otro día?', '[00:16.000 --> 00:18.000]  Sí.', '[00:18.000 --> 00:21.000]  ¿Usted le pudo avisar a ella?', '[00:21.000 --> 00:26.000]  Sí, pero me dice ella que ella está sin trabajo y que no puede hacer nada.', '[00:26.000 --> 00:30.000]  Pero tiene que comunicarse igual.', '[00:30.000 --> 00:31.000]  ¿Cómo?', '[00:31.000 --> 00:35.000]  Pero igual tiene que comunicarse mandando un whatsapp y yo las hago por teléfono.', '[00:35.000 --> 00:39.000]  Si estás sin trabajo, ¿tiene un número de teléfono para darme?', '[00:39.000 --> 00:41.000]  ¿Cómo, cómo?', '[00:41.000 --> 00:47.000]  ¿No tiene un número de teléfono donde yo puedo hablar con ella?', '[00:4

In [ ]:
df.Valida.dropna(inplace=True)

In [ ]:
df.Valida.value_counts(), df.Valida.value_counts().sum()

(1.0    626
 0.0    441
 9.0      3
 Name: Valida, dtype: int64, 1070)

In [ ]:
df2.Valida.value_counts(), df2.Valida.value_counts().sum()

(0    535
 1    504
 2     10
 `      1
 Name: Valida, dtype: int64, 1050)

In [ ]:
df.Valida = df.Valida.astype(str)
df2.Valida = df2.Valida.astype(str)

In [ ]:
df = df[df['Valida'].str.contains('1') != True]
df2 = df2[df2['Valida'].str.contains('0') == True]

In [ ]:
len(df), len(df2)

(1531, 535)

In [ ]:
df.head()

,ID_MOPC,IDMOROSO,IDCLIENTE,OPERADOR,CAMPAÑA,IDGRABACION,TIPO_DISCADOR,FECHA,ID,IDLLAMADA,...,LOG,DIRECCION,IDOPERADOR,AGENDAMIENTO_ESTADO,anos,mes,transcripcion_limpia,Contestador,transcripcion,Valida
0,242789,DNI 17446006,HSBC_LEGALES14,1893,13,0296415532961-58516182-20210412080350.mp3,Manual,2021-04-12,0FE83E97-4556-47AC-A378-BBD3F43E3B2F,58516182,...,|0|0296415532961|SIP/Telular35-0000003e|161822...,SALIENTE,JALTIMA,INTERESADO,2021,4,m ibs ...,0,"[00:00.000 --> 00:05.000] Hola, disculpe. Hab...",0.0
1,242828,DNI 22833374,BSR_TARDIA,1736,75,0341152104531-58516255-20210412083146.mp3,Manual,2021-04-12,50B44004-0532-4BC5-8E78-89BD1FFC98AA,58516255,...,|0|0341152104531|SIP/Telular21-000000bc|161822...,SALIENTE,MCARDOZO,INTERESADO,2021,4,si si que tal buen dia alejandra si ...,0,"['[00:00.000 --> 00:01.000] Hola.', '[00:01.0...",0.0
2,252660,DNI 14103422,BSR_TARDIA,3358,75,0352115465488-58539435-20210412145237.mp3,Manual,2021-04-12,6FB33DE6-0029-449A-AE8F-20CEF5D418E4,58539435,...,|0|0352115465488|SIP/Telular21-00009175|161825...,SALIENTE,KVISUÑA,INTERESADO,2021,4,que tal buen dia se encuentra si si c...,0,"['[00:00.000 --> 00:05.000] Hola. Hola, ¿qué ...",0.0
3,252704,DNI 24569175,BSR_TARDIA,15033,75,47033002-58539525-20210412145430.mp3,Manual,2021-04-12,EEBBA183-A9CF-4DC3-8B8A-A9FF7056ABAB,58539525,...,|0|47033002|DAHDI/220-78c|1618250889.44194|16:...,SALIENTE,ROBI,INTERESADO,2021,4,si buenos dias feliz carlos villegas no el...,0,['[00:00.000 --> 00:05.000] Hola. Hola. Hola....,nan
4,281844,DNI 32437164,FRANCES_2012,3456,30,1569041471-58596844-20210414110850.mp3,Manual,2021-04-14,2DFEA822-C269-437F-9BD1-16CB9B9A01BE,58596844,...,|0|1569041471|SIP/Telular26-0000353a|161841014...,SALIENTE,MCARDE,INTERESADO,2021,4,buenas dias se encuentra el senor juan nicolas...,0,"['[00:00.000 --> 00:02.000] Hola.', '[00:02.0...",nan


In [ ]:
df2.head()

,ID_MOPC,IDMOROSO,IDCLIENTE,OPERADOR,CAMPAÑA,IDGRABACION,TIPO_DISCADOR,FECHA,ID,IDLLAMADA,...,LOG,DIRECCION,IDOPERADOR,AGENDAMIENTO_ESTADO,anos,mes,transcripcion_limpia,transcripcion,Contestador,Valida
0,242789.0,DNI 17446006,HSBC_LEGALES14,1893.0,13.0,0296415532961-58516182-20210412080350.mp3,Manual,2021-04-12 00:00:00.000,0FE83E97-4556-47AC-A378-BBD3F43E3B2F,58516182.0,...,|0|0296415532961|SIP/Telular35-0000003e|161822...,SALIENTE,JALTIMA,INTERESADO,2021.0,4.0,m ibs ...,"[00:00.000 --> 00:05.000] Hola, disculpe. Hab...",0.0,0
1,242828.0,DNI 22833374,BSR_TARDIA,1736.0,75.0,0341152104531-58516255-20210412083146.mp3,Manual,2021-04-12 00:00:00.000,50B44004-0532-4BC5-8E78-89BD1FFC98AA,58516255.0,...,|0|0341152104531|SIP/Telular21-000000bc|161822...,SALIENTE,MCARDOZO,INTERESADO,2021.0,4.0,si si que tal buen dia alejandra si ...,"['[00:00.000 --> 00:01.000] Hola.', '[00:01.0...",0.0,0
2,252660.0,DNI 14103422,BSR_TARDIA,3358.0,75.0,0352115465488-58539435-20210412145237.mp3,Manual,2021-04-12 00:00:00.000,6FB33DE6-0029-449A-AE8F-20CEF5D418E4,58539435.0,...,|0|0352115465488|SIP/Telular21-00009175|161825...,SALIENTE,KVISU��,INTERESADO,2021.0,4.0,que tal buen dia se encuentra si si c...,"['[00:00.000 --> 00:05.000] Hola. Hola, ¿qué ...",0.0,0
3,252704.0,DNI 24569175,BSR_TARDIA,15033.0,75.0,47033002-58539525-20210412145430.mp3,Manual,2021-04-12 00:00:00.000,EEBBA183-A9CF-4DC3-8B8A-A9FF7056ABAB,58539525.0,...,|0|47033002|DAHDI/220-78c|1618250889.44194|16:...,SALIENTE,ROBI,INTERESADO,2021.0,4.0,si buenos dias feliz carlos villegas no el...,['[00:00.000 --> 00:05.000] Hola. Hola. Hola....,0.0,0
4,281844.0,DNI 32437164,FRANCES_2012,3456.0,30.0,1569041471-58596844-20210414110850.mp3,Manual,2021-04-14 00:00:00.000,2DFEA822-C269-437F-9BD1-16CB9B9A01BE,58596844.0,...,|0|1569041471|SIP/Telular26-0000353a|161841014...,SALIENTE,MCARDE,INTERESADO,2021.0,4.0,buenas dias se encuentra el senor juan nicolas...,"['[00:00.000 --> 00:02.000] Hola.', '[00:02.0...",0.0,0


In [ ]:
df = pd.concat([df, df2])

In [ ]:
# df = df[df.columns.drop(list(df.filter(regex='Unnamed')))]
# df.drop(columns='level_0', inplace=True)
df.reset_index(inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2066 entries, 0 to 2065
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 2066 non-null   int64  
 1   ID_MOPC               2066 non-null   float64
 2   IDMOROSO              2066 non-null   object 
 3   IDCLIENTE             2066 non-null   object 
 4   OPERADOR              2066 non-null   float64
 5   CAMPAÑA               2066 non-null   float64
 6   IDGRABACION           2066 non-null   object 
 7   TIPO_DISCADOR         2066 non-null   object 
 8   FECHA                 2066 non-null   object 
 9   ID                    2066 non-null   object 
 10  IDLLAMADA             2066 non-null   float64
 11  ESTADO                2066 non-null   object 
 12  SUBESTADO             2066 non-null   object 
 13  F_INICIO              2066 non-null   object 
 14  ANI_TELEFONO          2064 non-null   float64
 15  GRABACION_DEP        

In [ ]:
df.drop_duplicates().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2066 entries, 0 to 2065
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 2066 non-null   int64  
 1   ID_MOPC               2066 non-null   float64
 2   IDMOROSO              2066 non-null   object 
 3   IDCLIENTE             2066 non-null   object 
 4   OPERADOR              2066 non-null   float64
 5   CAMPAÑA               2066 non-null   float64
 6   IDGRABACION           2066 non-null   object 
 7   TIPO_DISCADOR         2066 non-null   object 
 8   FECHA                 2066 non-null   object 
 9   ID                    2066 non-null   object 
 10  IDLLAMADA             2066 non-null   float64
 11  ESTADO                2066 non-null   object 
 12  SUBESTADO             2066 non-null   object 
 13  F_INICIO              2066 non-null   object 
 14  ANI_TELEFONO          2064 non-null   float64
 15  GRABACION_DEP        

In [ ]:
df.Valida.unique()

array(['0.0', 'nan', '9.0', '0'], dtype=object)

In [ ]:
df = df[df['Valida'].str.contains('nan') != True]

In [ ]:
df.Valida.unique()

array(['0.0', '9.0', '0'], dtype=object)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 979 entries, 0 to 2065
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 979 non-null    int64  
 1   ID_MOPC               979 non-null    float64
 2   IDMOROSO              979 non-null    object 
 3   IDCLIENTE             979 non-null    object 
 4   OPERADOR              979 non-null    float64
 5   CAMPAÑA               979 non-null    float64
 6   IDGRABACION           979 non-null    object 
 7   TIPO_DISCADOR         979 non-null    object 
 8   FECHA                 979 non-null    object 
 9   ID                    979 non-null    object 
 10  IDLLAMADA             979 non-null    float64
 11  ESTADO                979 non-null    object 
 12  SUBESTADO             979 non-null    object 
 13  F_INICIO              979 non-null    object 
 14  ANI_TELEFONO          978 non-null    float64
 15  GRABACION_DEP         

In [ ]:
transcripciones = df["transcripcion_limpia"]

In [ ]:
df_columns = df.columns.to_list()

In [ ]:
# dfff2 = df2.columns.to_list()

In [ ]:
# l3 = [x for x in dfff2 if x not in df_columns]

In [ ]:
# l3

In [ ]:
# l3 = ['Saludo',
#  'Oferta',
#  'Aceptacion',
#  'Whatssapp',
#  'Modo_pago',
#  'Trabajando',
#  'Transferencia',
#  'Paga',
#  'Despedida',
#  'Trabajo',
#  'Tarjeta',
#  'Deuda',
#  'Reclamo',
#  'Ayudar',
#  'Asentado',
#  'Renovacion',
#  'Pago',
#  'Refinanciacion',
#  'Cumplir',
#  'Voluntad',
#  'Agendar',
#  '`Trabajo',
#  'Acuerdo',
#  'Hablar',
#  'Entidades',
#  'Bonificacion',
#  'Resolver',
#  'Baja',
#  'Ayuda',
#  'Posibilidades',
#  'Renovar',
#  'Refinanciar',
#  'Posibilidad',
#  'Numero',
#  'Depositar',
#  'Comunico',
#  'Saldo',
#  'Cuota',
#  'Pauta',
#  'Banco',
#  'Utilizar',
#  'Cancela',
#  'Sistemas',
#  'Libre_deuda',
#  'Pendiente',
#  'Acreditado',
#  'Deposito',
#  'Problema',
#  'Informacion',
#  'Autorizacion',
#  'Metodo_pago',
#  'Email',
#  'Reactivacion',
#  'Datos',
#  'Contacto',
#  'Vacuna',
#  'Descuento',
#  'Opcion',
#  'Cuotas',
#  'Esfuerzo',
#  'Intencion',
#  'Grabar',
#  'Negociacion',
#  'Turno',
#  'Propuesta',
#  'Solucion',
#  'Gracias',
#  'Manejar',
#  'Solicitud',
#  'Vencimiento',
#  'Seguimiento',
#  'Comprobante',
#  'Plan',
#  'Pagar',
#  'Abonar',
#  'luego',
#  'Confirmado',
#  'Contactar',
#  'Cbu',
#  'Estafa',
#  'Llamadas',
#  'Telefono',
#  'Dinero',
#  'Ofrecer',
#  'Confirmar',
#  'Ordenar',
#  'Esperar',
#  'Llamar',
#  'Inconveniente',
#  'Llamado',
#  'Correo',
#  'Regularizar',
#  'Mensaje',
#  'Atender',
#  'Registro',
#  'Interes',
#  'Financiar',
#  'Banking',
#  'Usuario',
#  'Sucursal',
#  'Enviar',
#  'Abogado',
#  'Legales',
#  'Situacion',
#  'Cobrar',
#  'Retraso',
#  'Contestacion',
#  'Negativo',
#  'Atraso',
#  'Aislamiento',
#  'Semana',
#  'Beneficio',
#  'Cancelar',
#  'Cuenta',
#  'Informar',
#  'Plata',
#  'Judicial',
#  'Credito',
#  'Asesorar',
#  'Cobranza',
#  'Duda',
#  'Mopc',
#  'Laburo',
#  'Localizar',
#  'Plazo',
#  'Cortar',
#  'Legajo',
#  'Honorarios',
#  'Tiempo',
#  'Comunicar',
#  'Recordatorio',
#  'Avisar',
#  'Internet',
#  'Arreglar',
#  'Resumen',
#  'Clave',
#  'Importe',
#  'Pandemia',
#  'Familia',
#  'Intereses',
#  'Analizar']

In [ ]:
# from gensim.models import Word2Vec
# word2vec = Word2Vec(sentences=transcripciones)


In [ ]:
# word2vec.wv.most_similar('familia', topn=10)

In [ ]:
# import gensim.downloader as api
# word_vectors = api.load("glove-wiki-gigaword-100")  # load pre-trained word-vectors from gensim-data
# # Check the "most similar words", using the default "cosine similarity" measure.
# # result = word_vectors.most_similar('hola')

In [ ]:
# word_vectors.most_similar('saludo')

In [ ]:
# result

In [ ]:
# from gensim.models import KeyedVectors
# word_vectors.save('vectors.kv')
# reloaded_word_vectors = KeyedVectors.load('vectors.kv', mmap='r')

In [ ]:
# reloaded_word_vectors.most_similar('oferta')

In [ ]:
# for n in range(10):
#   try:
#     print(f''+ str(l3[n]) + ': \n' + str(reloaded_word_vectors.most_similar(l3[n].lower())))
#   except:
#     KeyError

In [ ]:
# import requests
# from bs4 import BeautifulSoup
# url='http://www.wordreference.com/sinonimos/'
# for n in range(10):
#   enlace=l3[n].replace('_','')
#   buscar=url+enlace
#   resp=requests.get(buscar)
#   bs=BeautifulSoup(resp.text,'lxml')
#   lista=bs.find_all(class_='trans clickable')
#   for sin in lista:
#       sino=sin.find_all('li')
#       for fin in sino:
#           print(f''+ str(enlace) + ': \n' + str(fin.next_element))

In [ ]:
# palabras = []
# for transcripcion in transcripciones:
#     palabras.extend(nltk.word_tokenize(transcripcion))

# fdist = nltk.FreqDist(palabras)
# palabras_clave = fdist.most_common(10)

# from sklearn.decomposition import LatentDirichletAllocation

# vectorizer = CountVectorizer()
# X = vectorizer.fit_transform(transcripciones)
# lda = LatentDirichletAllocation(n_components=10)
# lda.fit(X)

# temas = []
# for topic in lda.components_:
#     tema = " ".join([vectorizer.get_feature_names()[i] for i in topic.argsort()[:-10 - 1:-1]])
#     temas.append(tema)

In [ ]:
new_stopwords = ['si',
 'bueno',
 'banco',
 'usted',
 'tardes',
 'buenas',
#  'pago',
#  'deuda',
 'bien',
 'mas',
 'dias',
 'hacer',
 'senor',
 'ver',
 'puedo',
#  'cuotas',
 'va',
 'gracias',
 'perfecto',
 'puede',
 'ahora',
 'voy',
 'dia',
 'entonces',
 'buenos',
 'claro',
 'numero',
 'senora',
 'ah',
 'ok',
 'asi',
 'tal',
 'ahi',
 'favor',
 'manana',
 'luego',
 'estudio',
#  'pagar',
 'dale',
 'mes',
 'plan',
 'hablar',
 'agencia',
 'hola',
 'cómo',
 'ahí',
 'así',
 'día',
 'como',
 'ahi',
 'asi',
 'dia']

In [ ]:
def get_keywords(columna):
  stpwrd = nltk.corpus.stopwords.words('spanish')
  stpwrd.extend(new_stopwords)
  vectorizer = TfidfVectorizer(stop_words=stpwrd)
  X = vectorizer.fit_transform(columna)
  features = vectorizer.get_feature_names()
  scores = X.toarray().sum(axis=0)
  keywords = [features[i] for i in scores.argsort()[::-1][:20]]
  return keywords

In [ ]:
get_keywords(df['transcripcion_limpia'])

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['pago',
 'deuda',
 'cuotas',
 'pesos',
 'whatsapp',
 'pagar',
 'tarjeta',
 'cuenta',
 'hoy',
 'saldo',
 'dos',
 'descuento',
 'mil',
 'tres',
 'despues',
 'pueda',
 'cuota',
 'telefono',
 'semana',
 'poder']

In [ ]:
def palbras_mas_importantes(turnos):
    # Tokenizamos los turnos de habla
    tokenizer = RegexpTokenizer(r'\w+')
    df['turnos_tokenizados'] = turnos.apply(lambda x: tokenizer.tokenize(x))

    # Eliminamos las stopwords
    stpwrd = nltk.corpus.stopwords.words('spanish')
    stpwrd.extend(new_stopwords)
    df['turnos_filtrados'] = df['turnos_tokenizados'].apply(lambda x: [word for word in x if word not in stpwrd])

    # Lematizamos los turnos de habla
    lemmatizer = WordNetLemmatizer()
    df['turnos_lematizados'] = df['turnos_filtrados'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

    # Calculamos la longitud promedio de los turnos de habla
    df['longitud_turnos'] = df['turnos_lematizados'].apply(lambda x: len(x))
    longitud_promedio = df['longitud_turnos'].mean()

    # Calculamos los temas principales
    palabras = []
    for turno in df['turnos_lematizados']:
        palabras.extend(turno)
    frecuencia = pd.Series(palabras).value_counts()
    palbras_mas_importantes = frecuencia[:10]

    # Calculamos las palabras clave
    palabras_clave = []
    for turno in df['turnos_lematizados']:
        for palabra in turno:
            if palabra not in palabras_clave and palabra not in stpwrd:
                palabras_clave.append(palabra)
    return palbras_mas_importantes, palabras

In [ ]:
# palbras_mas_importantes(df['transcripcion_limpia'])

In [ ]:
transcripciones = palbras_mas_importantes(df['transcripcion_limpia'])[1]

In [ ]:
transcripciones

['m',
 'ibs',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mib',
 'mm',
 'mi

In [ ]:
# def obtener_palabras_clave(transcripciones):
#     from rake_nltk import Rake
#     r = Rake()
#     keywords = []
#     for transcripcion in transcripciones:
#         r.extract_keywords_from_text(transcripcion)
#         keywords.append(r.get_ranked_phrases())
#     return keywords

In [ ]:
!python -m spacy download es_dep_news_trf
import spacy

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-01-30 12:58:45.489825: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.2/410.2 MB 3.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_dep_news_trf')


In [ ]:
!pip install spacy-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def identify_topics(dialogs, n_topics=5, n_words=10):
    # Procesamiento del texto con spaCy
    nlp = spacy.load('es_dep_news_trf')
    docs = [nlp(dialog) for dialog in dialogs]
    
    # Construcción del diccionario y la matriz de términos
    stpwrd = nltk.corpus.stopwords.words('spanish')
    stpwrd.extend(new_stopwords)
    cv = CountVectorizer(stop_words=stpwrd)
    word_count = cv.fit_transform(dialogs)
    word_list = cv.get_feature_names()
    
    # Entrenamiento del modelo LDA
    lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
    lda.fit(word_count)
    
    # Extracción de los tópicos principales
    top_topics = []
    for topic_idx, topic in enumerate(lda.components_):
        top_words = [word_list[i] for i in topic.argsort()[:-n_words - 1:-1]]
        top_topics.append(top_words)
    
    return top_topics


In [ ]:
# identify_topics(transcripciones)

In [ ]:
temas = ['pago',
 'suscribete',
 'deuda',
 'mensaje',
 'telefono',
 'pesos',
 'whatsapp',
 'cuotas',
 'pagar',
 'hoy',
 'tarjeta',
 'familia',
 'trabajo']

In [ ]:
def obtener_frase(df):
  filas = []

  # Itera sobre cada fila del dataframe
  for index, row in compromiso.iterrows():
    # Extrae la lista de frases de la fila
    frases = row['frases']
    
    # Crea un diccionario vacío para almacenar las frases por tema para la fila actual
    frases_por_tema = {}
    
    # Itera sobre cada tema
    for tema in temas:
      # Itera sobre cada frase
      for frase in frases:
        # Comprueba si la frase contiene el tema actual
        if tema in frase:
          # Añade la frase al diccionario si no existe aún una lista para el tema
          if tema not in frases_por_tema:
            frases_por_tema[tema] = []
          # Añade la frase a la lista del diccionario correspondiente al tema
          frases_por_tema[tema].append(frase)
    
    # Añade el diccionario a la lista de diccionarios
    filas.append(frases_por_tema)

    #Transformalo a Dataframe 
  frases_claves = pd.DataFrame.from_records(filas)

  return frases_claves

In [ ]:
compromiso = df[df['AGENDAMIENTO_ESTADO'].str.contains('COMPROMISO') == True]
# compromiso.drop(columns='level_0', inplace=True)
compromiso.reset_index(inplace=True)

In [ ]:
compromiso['frases'] = None
for n in range(len(compromiso)):
  compromiso['frases'][n] = re.split(r'\s{2,}', compromiso.transcripcion_limpia[n])

<ipython-input-57-65e6958bc711>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compromiso['frases'] = None
<ipython-input-57-65e6958bc711>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compromiso['frases'][n] = re.split(r'\s{2,}', compromiso.transcripcion_limpia[n])


In [ ]:
prueba = obtener_frase(compromiso)
prueba.head()

,pago,deuda,pesos,whatsapp,pagar,telefono,tarjeta,cuotas,familia,hoy,trabajo,mensaje,suscribete
0,[el pago tiene que estar antes del mediodia si],[bien la deuda que presenta el dia de la fecha...,[pesos],[yo le voy a enviar tiene whatsapp en este num...,[perfecto pero lo puedo pagar el martes que vi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[si senor como verifico que usted ya pago la d...,[si senor como verifico que usted ya pago la d...,[pesos si tengo que hacerle nuevamente un plan...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"[eso te queria decir porque ella pago, o sino ...",[la deuda que presenta a usted o su esposo es ...,NaN,NaN,NaN,[este telefono se lo le dan a mi marido que ti...,[este telefono se lo le dan a mi marido que ti...,NaN,NaN,NaN,NaN,NaN,NaN
3,[para cuando podriamos agendar otra vez su pag...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[mientras yo espero el pago la acreditacion de...,NaN,[si senora soy yo ayer ya se le depositaron ot...,NaN,[claro que ayer se pagaron cinco mil mas bueno...,NaN,[ah ok bueno si jacobo los cinco mil anteriore...,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
prueba.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   pago        221 non-null    object
 1   deuda       161 non-null    object
 2   pesos       150 non-null    object
 3   whatsapp    151 non-null    object
 4   pagar       142 non-null    object
 5   telefono    69 non-null     object
 6   tarjeta     82 non-null     object
 7   cuotas      83 non-null     object
 8   familia     26 non-null     object
 9   hoy         112 non-null    object
 10  trabajo     73 non-null     object
 11  mensaje     66 non-null     object
 12  suscribete  1 non-null      object
dtypes: object(13)
memory usage: 26.5+ KB


In [ ]:
compromiso.frases[0]

['si maria me escucha este corton',
 'si le escucho le escucho',
 'bien la deuda que presenta el dia de la fecha son',
 'pesos',
 'y solo que le puedo ofrecer para que cancele con el descuento de este mes de abril',
 'que vence el martes que viene es un descuento mas',
 'no manana',
 'no manana el otro el otro martes',
 'es un descuento que le puedo dar del',
 'casi',
 'con',
 'cancelan la deuda',
 'perfecto pero lo puedo pagar el martes que viene',
 'si usted le mando un link o algun codigo o algo para que yo pueda pagar',
 'yo le voy a enviar tiene whatsapp en este numero',
 'si',
 'bueno yo le voy a enviar un whatsapp',
 'el pago tiene que estar antes del mediodia si',
 'a mas tardar a las',
 'horas porque en ese horario hay que entregar los tickets al banco si',
 'ahorita la voy a agregar al whatsapp y le voy a pasar el numero de cuenta donde tiene que pagar',
 'es una cuenta corriente o un cbu',
 'puede ir a cualquier santander con la cuenta corriente',
 'o si no hace una transfer

In [ ]:
def analisis_sentimientos_bert(df):
  # Crea una nueva columna en el dataframe para almacenar el resultado del análisis de sentimientos
  df['sentimiento'] = None

  # Crea una instancia del tokenizador de BERT que soporte el idioma español
  tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

  # Carga el modelo pre-entrenado de BERT para clasificación de secuencias que soporte el idioma español
  model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=4)

  # Itera sobre cada fila del dataframe
  for index, row in df.iterrows():
    # Obtiene el texto de la columna transcripcion_limpia de la fila actual
    texto = row['frases']

    # Tokeniza el texto y lo convierte en un tensor de PyTorch
    input_ids = torch.tensor(tokenizer.encode(texto, add_special_tokens=True)).unsqueeze(0)  # agrega dos dimensiones adicional para hacerlo compatible con el modelo BERT

    # Ejecuta el modelo BERT para clasificar la secuencia de tokens
    logits = model(input_ids)[0]

    # Obtiene la probabilidad de cada clase (positiva y negativa)
    probabilidades = torch.nn.functional.softmax(logits, dim=0)

    # Obtiene la clase con mayor probabilidad
    clase_predicha = probabilidades.argmax().item()

# Asigna el sentimiento a la columna sentimiento del dataframe según la clase predicha
    if clase_predicha == 0:
      df.loc[index, 'sentimiento'] = 'positivo'
    else:
      df.loc[index, 'sentimiento'] = 'negativo'

In [ ]:
df.drop(0, inplace=True)

In [ ]:
df.reset_index(inplace=True)

In [ ]:
df['frases'] = None
for n in range(len(df)):
  df['frases'][n] = re.split(r'\s{2,}', str(df.transcripcion_limpia[n]))

<ipython-input-69-1b634f6c50f9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['frases'][n] = re.split(r'\s{2,}', str(df.transcripcion_limpia[n]))


In [ ]:
analisis_sentimientos_bert(df)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [ ]:
compromiso.head()

In [ ]:
compromiso.longitud_turnos.value_counts()

In [ ]:
df.Valida.value_counts()

In [ ]:
df.sentimiento.value_counts() # df negativo    1432 positivo     844

negativo    740
positivo    238
Name: sentimiento, dtype: int64

In [ ]:
df.to_csv('df_final.csv')

In [ ]:
positivos = df[df['sentimiento'].str.contains('positivo') == True]
negativo = df[df['sentimiento'].str.contains('negativo') == True]

In [ ]:
compromisos = positivos[positivos['AGENDAMIENTO_ESTADO'].str.contains('COMPROMISO') == True]

In [ ]:
compromisos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67 entries, 32 to 964
Data columns (total 36 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   level_0               67 non-null     int64  
 1   index                 67 non-null     int64  
 2   ID_MOPC               67 non-null     float64
 3   IDMOROSO              67 non-null     object 
 4   IDCLIENTE             67 non-null     object 
 5   OPERADOR              67 non-null     float64
 6   CAMPAÑA               67 non-null     float64
 7   IDGRABACION           67 non-null     object 
 8   TIPO_DISCADOR         67 non-null     object 
 9   FECHA                 67 non-null     object 
 10  ID                    67 non-null     object 
 11  IDLLAMADA             67 non-null     float64
 12  ESTADO                67 non-null     object 
 13  SUBESTADO             67 non-null     object 
 14  F_INICIO              67 non-null     object 
 15  ANI_TELEFONO          6

In [ ]:
positivos.to_csv('positivos.csv')
from google.colab import files
files.download('/content/positivos.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
negativo.to_csv('negativo.csv')
from google.colab import files
files.download('/content/negativo.csv')